In [1]:
import datasets as ds

human_eval = ds.load_dataset("openai/openai_humaneval")

print(human_eval)

DatasetDict({
    test: Dataset({
        features: ['task_id', 'prompt', 'canonical_solution', 'test', 'entry_point'],
        num_rows: 164
    })
})


In [3]:
human_eval["test"].to_csv("human_eval.csv", index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

196170

In [11]:
''' Rename all the functions in the dataset to "test"
'''
import re

def lambda_rename (row: dict):
    fun_name: str = row["entry_point"]
    rename = "test"
    regex_pattern = rf"(def |>>> )({fun_name})(\()"
    replacement_str = rf"\1{rename}\3"

    regex_id = r"HumanEval"
    replacement_id = r"HumanEvalRenamed"

    renamed_id = re.sub(regex_id, replacement_id, row["task_id"])
    renamed_prompt = re.sub(regex_pattern, replacement_str, row["prompt"])
    row["task_id"] = renamed_id
    row["entry_point"] = rename
    row["prompt"] = renamed_prompt
    return row

renamed_set = human_eval["test"].map(lambda_rename)

human_eval["renamed"] = renamed_set

print(human_eval)

DatasetDict({
    test: Dataset({
        features: ['task_id', 'prompt', 'canonical_solution', 'test', 'entry_point'],
        num_rows: 164
    })
    renamed: Dataset({
        features: ['task_id', 'prompt', 'canonical_solution', 'test', 'entry_point'],
        num_rows: 164
    })
})


In [12]:
human_eval["renamed"].to_csv("human_eval_renamed.csv", index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

193303